In [3]:
pip install ccxt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import plotly.graph_objects as go
from datetime import timedelta
import plotly.io as pio
import requests
from itertools import combinations

In [2]:
binance = ccxt.binance()
okx = ccxt.okex()
bitstamp = ccxt.bitstamp()
bitfinex = ccxt.bitfinex()
kraken = ccxt.kraken()


In [3]:
# from datetime import datetime as dt, timedelta
# import ccxt

# Convert a date to a timestamp in milliseconds
def date_to_milliseconds(date_str):
    epoch = dt.utcfromtimestamp(0)
    dt_obj = dt.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int((dt_obj - epoch).total_seconds() * 1000)

def get_data(exchange, symbol, start_dt, end_dt, interval):     # interval inputted as a timedelta
    start_dt_str = start_dt.strftime("%Y-%m-%d %H:%M:%S")
    since = date_to_milliseconds(start_dt_str)
    duration = end_dt - start_dt
    limit = int(duration / interval)
    
    # Extract interval in the format ccxt expects (like '30m', '1h', etc.)
    minutes = interval.total_seconds() / 60
    if minutes == 60:
        timeframe = '1h'
    else:
        timeframe = '{}m'.format(int(minutes))
        
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
    except:
        'No symbol found on this exchange'
    return ohlcv

# symbol = 'BTC/USDT'
# start_dt = dt(2023, 8, 19)
# end_dt = dt(2023, 8, 29)

# exchange1_data = get_data(exchange=bitstamp, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=60))
# exchange2_data = get_data(exchange=bitfinex, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=60))

# print(exchange1_data[0])
# print('')
# print(exchange2_data[0])

x1 = get_data(binance,'BTC/USDT',dt(2023,8,20),dt(2023,8,30),timedelta(minutes = 60))
x2 = get_data(okx,'BTC/USDT',dt(2023,8,20),dt(2023,8,30),timedelta(minutes = 60))

In [4]:
def to_df_price(data):
    unix, op, hi, lo, cl, vol = [], [], [], [], [], []
    for i in data:
        unix.append(i[0])
        op.append(i[1])
        hi.append(i[2]) 
        lo.append(i[3]) 
        cl.append(i[4]) 
        vol.append(i[5])


    # converting unix to python datetime:
    def to_date_time(unix):
        dt_object = dt.fromtimestamp(unix/1000)
        return dt_object


    date_time = list(map(to_date_time,unix))

    
    dic = {'Date_Time':date_time,'Open': op,'High':hi,'Low': lo,'Close':cl,'Volume':vol}
    df = pd.DataFrame(dic)
    
    df['Times']=df['Date_Time'].dt.time
    df['Dates'] = df['Date_Time'].dt.date
    df['Vol$'] = df['Volume']*df['Close']
#     df = df.groupby('Times',as_index = False)['Vol$'].mean()
#     df.groupby('Times').agg({'Vol$':'mean','Close':'last'}).reset_index()
    df['Cum_vol$'] = df['Vol$'].cumsum()
    
    return df

# exchange1_price = to_df_price(exchange1_data)
# exchange2_price = to_df_price(exchange2_data)

# print(exchange1_price['Close'])
# print('')
# print(exchange2_price['Close'])

In [5]:
# y1 = to_df_price(x1)
# y2 = to_df_price(x2)

In [6]:
# print(exchange1_price.iloc[2])
# print('')
# print(exchange2_price.iloc[2])
# compare_p

In [7]:
def compare(ex1,ex2):
    compare_p = pd.DataFrame()
    compare_p['Date_Time'] = ex1['Date_Time']
    compare_p['Dates'] = ex2['Dates']
    compare_p['percentage_diff'] = 100*(ex1['Close']-ex2['Close'])/((ex1['Close']+ex2['Close'])/2)
    compare_p['price_diff'] = (ex1['Close']-ex2['Close'])
    
    compare_p = compare_p[compare_p['percentage_diff']<=5]
    compare_p = compare_p.groupby('Dates').max()
    
    return compare_p['percentage_diff'].mean(),compare_p['price_diff'].mean()
#     return compare_p



In [8]:
# compare(y1,y2)

In [11]:
def get_tickers():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    response = requests.get(url, params={
            "vs_currency": "usd",
            "order": "market_cap_desc",
            "limit": 21,
            "sparkline": "false",
            "market_data": "true",
            "price_change_percentage": "false",
            "market_cap_change_percentage": "false",
            "exchange_ids": "binance",  # filtering coins that are listed on Binance
            "exchange_limit": "false",
         # this filters by Ethereum-based tokens; remove if you want all tokens regardless of platform
            "liquidity_order": "true",  # this can help ensure that the tokens returned have sufficient liquidity
            "lang": "en",
        })
    data = response.json()

    tokens = [(coin['id'], coin['market_cap']) for coin in data]

    # print(tokens[0][0])
    names = []
    for i in range(0,21):
        names.append(tokens[i][0])

    # print(names)

    standardized_names = {
        'bitcoin': 'BTC',
        'ethereum': 'ETH',
        'tether': 'USDT',
        'binancecoin': 'BNB',
        'ripple': 'XRP',
        'usd-coin': 'USDC',
        'staked-ether': 'STETH',  
        'cardano': 'ADA',
        'dogecoin': 'DOGE',
        'solana': 'SOL',
        'tron': 'TRX',
        'the-open-network': 'TON',  
        'polkadot': 'DOT',
        'matic-network': 'MATIC',
        'litecoin': 'LTC',
        'shiba-inu': 'SHIB',
        'wrapped-bitcoin': 'WBTC',
        'bitcoin-cash': 'BCH',
        'dai': 'DAI',
        'avalanche-2': 'AVAX',
        'leo-token':'LEO'              
    }
    tickers = []
    for i in names:
        try:
            tickers.append(standardized_names[i]+'/USDT')
        except:
            continue
    tickers.remove('USDT/USDT')
    return tickers

# tickers

In [12]:
get_tickers()

['BTC/USDT',
 'ETH/USDT',
 'BNB/USDT',
 'XRP/USDT',
 'USDC/USDT',
 'STETH/USDT',
 'ADA/USDT',
 'DOGE/USDT',
 'SOL/USDT',
 'TRX/USDT',
 'TON/USDT',
 'DOT/USDT',
 'MATIC/USDT',
 'LTC/USDT',
 'SHIB/USDT',
 'WBTC/USDT',
 'DAI/USDT',
 'BCH/USDT',
 'AVAX/USDT']

In [13]:
def get_table(start_dt,end_dt,exchange1,exchange2):
    
    tickers = get_tickers()
    data = []
#     tickers = ['BTC/USDT']
    for symbol in tickers:
        try:
            exchange1_data = get_data(exchange=exchange1, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=60))
            exchange2_data = get_data(exchange=exchange2, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=60))
            exchange1_price = to_df_price(exchange1_data)
            exchange2_price = to_df_price(exchange2_data)
#             print(exchange1_price.iloc[0])
#             print('')
#             print(exchange2_price.iloc[0])
    #         print(compare(exchange1_price,exchange2_price))
            data.append((compare(exchange1_price,exchange2_price),symbol))
        except:
            tickers.remove(symbol)
    
    symbol = [i[1] for i in data]
    max_percentage = [i[0][0] for i in data]
    max_dollar = [i[0][1] for i in data]

    dic = {'symbol':symbol,'Max_percentage_differnce':max_percentage,'Max_dollar_difference':max_dollar}

    table = pd.DataFrame(dic)
    print(str(exchange1)+'/'+str(exchange2))
    return table


In [14]:
start_dt = dt(2023, 8, 19)
end_dt = dt(2023, 8, 29)
# print(get_table(start_dt,end_dt,binance,bitstamp))

exhanges = [binance,okx,bitstamp,bitfinex,kraken]
exchange_combos = list(combinations(exhanges, 2))

# print(exchange_combos)

for i in exchange_combos:
    print(get_table(start_dt,end_dt,i[0],i[1]))
    print('')
    

Binance/OKX
        symbol  Max_percentage_differnce  Max_dollar_difference
0     BTC/USDT                  0.015075           3.938182e+00
1     ETH/USDT                  0.011035           1.827273e-01
2     BNB/USDT                  0.074767           1.609091e-01
3     XRP/USDT                  0.041386           2.172727e-04
4    USDC/USDT                  0.009086           9.090909e-05
5    DOGE/USDT                  0.039020           2.454545e-05
6     SOL/USDT                  0.060580           1.263636e-02
7     TRX/USDT                  0.025004           1.909091e-05
8   MATIC/USDT                  0.057204           3.181818e-04
9     LTC/USDT                  0.041994           2.727273e-02
10   SHIB/USDT                  0.108144           8.818182e-09
11   WBTC/USDT                  0.261606           6.827636e+01
12   AVAX/USDT                  0.099683           1.018182e-02

Binance/Bitstamp
      symbol  Max_percentage_differnce  Max_dollar_difference
0   BTC/USDT

In [ ]:
ob = (binance.fetch_order_book('BTC/USDT',limit = 1))
ob['bids'][0]

[25983.9, 8.9625]

In [ ]:
def unix_to_datetime(unix_milliseconds):
    # Convert milliseconds to seconds
    unix_seconds = unix_milliseconds / 1000.0
    # Convert Unix timestamp to datetime object
    date_time = dt.utcfromtimestamp(unix_seconds) + timedelta(hours = 1)
    return date_time

unix_to_datetime(1693568894)

datetime.datetime(1970, 1, 20, 15, 26, 8, 894000)

In [ ]:
import time
delay = 60
# for i in range(100):
#     print (pd.DataFrame(binance.fetch_order_book ('BTC/USDT',limit = 2)))
#     time.sleep(delay) # rate limit



# dic = {'symbol':symbol,'bid':bid,'ask':ask,'datetime'}

symbol,bid,ask,datetime = [],[],[],[]

for i in range(10):
    ob = binance.fetch_order_book ('BTC/USDT',limit = 1)
    symbol.append(ob['symbol'])
    bid.append(ob['bids'])
    ask.append(ob['asks'])
    datetime.append(dt.now())
    time.sleep(60)
    


# df = pd.DataFrame(binance.fetch_order_book ('BTC/USDT',limit = 1))
# print(df)
# df = df[['symbol','bids','asks']]

NameError: name 'binance' is not defined

,symbol,bid,ask,datetime
0,BTC/USDT,"[[26021.82, 3.03299]]","[[26021.83, 20.25625]]",2023-09-01 11:26:37.321981
1,BTC/USDT,"[[26020.8, 0.62314]]","[[26020.81, 27.86678]]",2023-09-01 11:27:37.636019
2,BTC/USDT,"[[26017.02, 2.45735]]","[[26017.03, 18.87532]]",2023-09-01 11:28:37.888681


In [ ]:
df

,symbol,bids,asks,timestamp,datetime,nonce
0,BTC/USDT,"[26009.8, 0.18799074]","[26009.9, 1.8654531]",1693559604408,2023-09-01T09:13:24.408Z,None
1,BTC/USDT,"[26008.7, 0.04345232]","[26010.0, 0.186]",1693559604408,2023-09-01T09:13:24.408Z,None
2,BTC/USDT,"[26008.1, 0.11160001]","[26010.4, 0.00115339]",1693559604408,2023-09-01T09:13:24.408Z,None
3,BTC/USDT,"[26008.0, 0.186]","[26010.6, 0.03067465]",1693559604408,2023-09-01T09:13:24.408Z,None
4,BTC/USDT,"[26007.7, 0.09228036]","[26010.7, 0.28846626]",1693559604408,2023-09-01T09:13:24.408Z,None
5,BTC/USDT,"[26007.1, 0.00529453]","[26010.8, 0.00288485]",1693559604408,2023-09-01T09:13:24.408Z,None
6,BTC/USDT,"[26006.9, 0.30000009]","[26011.7, 0.05385]",1693559604408,2023-09-01T09:13:24.408Z,None
7,BTC/USDT,"[26006.6, 0.425]","[26012.0, 0.186]",1693559604408,2023-09-01T09:13:24.408Z,None
8,BTC/USDT,"[26006.3, 1.51671148]","[26012.4, 0.01193911]",1693559604408,2023-09-01T09:13:24.408Z,None
9,BTC/USDT,"[26006.1, 2.94747925]","[26012.5, 0.04]",1693559604408,2023-09-01T09:13:24.408Z,None
